In [ ]:
pip install pypdf

In [ ]:
from openai import AzureOpenAI
import os
from IPython.display import  Markdown

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.environ.get("OPENAI_KEY"),
  api_version="2023-12-01-preview"
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory


In [ ]:
txt_file_path = '../docs/Phone-List-Schools.pdf'
loader = PyPDFLoader(file_path=txt_file_path)
data = loader.load()

In [ ]:
data

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
embeddings = AzureOpenAIEmbeddings(api_key=os.environ.get("OPENAI_KEY") ,api_version= "2023-12-01-preview",azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),azure_deployment="text-embedding-ada-002")

In [ ]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:
llm = AzureChatOpenAI(model="gpt-4",api_key=os.environ.get("OPENAI_KEY") ,api_version= "2023-12-01-preview",azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"))

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [ ]:
query = "Redmond school location and phone number?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

In [ ]:
query = "List all the schools in Redmond in pointers"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

In [34]:
query = "List all the schools starting with letter F"
result = conversation_chain({"question": query})
answer = result["answer"]
answer
Markdown(answer)

Certainly! Here are the schools in Redmond that start with the letter "F" as per the provided context:

1. Frost - 11801 NE 140th Street, Kirkland 98034 - Phone: 936-2560
2. Futures School - 10601 NE 132nd Street, Kirkland 98034 (at JHS) - Phone: 936-1635

Please note that Frost is listed under Kirkland but is included here as it may serve the Redmond area and is part of the Lake Washington School District. Futures School is listed at a Kirkland address but is included due to potential service to the Redmond community.